In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import pycountry
from countryinfo import CountryInfo as CInfo 
import matplotlib.pyplot as plt
import  altair  as  alt 
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [51]:
# Step 1: Load and preprocess the data
url = "https://raw.githubusercontent.com/github/innovationgraph/main/data/languages.csv"
data = pd.read_csv(url, delimiter=',', keep_default_na=False)
data

,num_pushers,language,language_type,iso2_code,year,quarter
0,2066,HTML,markup,AE,2020,1
1,1627,CSS,markup,AE,2020,1
2,288,Jupyter Notebook,markup,AE,2020,1
3,108,Vue,markup,AE,2020,1
4,1734,JavaScript,programming,AE,2020,1
...,...,...,...,...,...,...
104437,198,TypeScript,programming,ZW,2023,4
104438,150,C,programming,ZW,2023,4
104439,140,Dockerfile,programming,ZW,2023,4
104440,140,PHP,programming,ZW,2023,4


In [52]:
# Step 2: Replace empty strings in 'iso2_code' with 'NA'
data['iso2_code'].fillna('NA', inplace=True)

# Filter out EU and XK
data = data[(data.iso2_code != "EU") & (data.iso2_code != "XK")]

# Step 3: Convert 'num_pushers' to numeric, coerce errors to NaN
data['num_pushers'] = pd.to_numeric(data['num_pushers'], errors='coerce')

# Create 'year_quarter' column
data['year_quarter'] = data['year'].astype(str) + '-Q' + data['quarter'].astype(str)

# Create a unique identifier
data['unique_id'] = data['iso2_code'] + '-' + data['language']

# Drop unnecessary columns
data = data.drop(columns=['language_type', 'language', 'iso2_code', 'year', 'quarter'], errors='ignore')

# Step 4: Balance the panel data
# Extract unique year_quarters
year_quarters = sorted(data['year_quarter'].unique())

# Create a MultiIndex with all combinations of unique_id and year_quarter
multi_index = pd.MultiIndex.from_product([data['unique_id'].unique(), year_quarters], names=['unique_id', 'year_quarter'])

# Reindex the data to create a balanced panel
balanced_df = data.set_index(['unique_id', 'year_quarter']).reindex(multi_index).reset_index()

# Step 5: Fill missing values in 'num_pushers' with 0
balanced_df['num_pushers'] = balanced_df['num_pushers'].fillna(0)

# Display the balanced panel data
balanced_df

,unique_id,year_quarter,num_pushers
0,AE-HTML,2020-Q1,2066.0
1,AE-HTML,2020-Q2,3254.0
2,AE-HTML,2020-Q3,3208.0
3,AE-HTML,2020-Q4,3255.0
4,AE-HTML,2021-Q1,3321.0
...,...,...,...
136203,YE-TypeScript,2022-Q4,0.0
136204,YE-TypeScript,2023-Q1,0.0
136205,YE-TypeScript,2023-Q2,0.0
136206,YE-TypeScript,2023-Q3,0.0


In [53]:
# Split 'unique_id' into 'iso2_code' and 'language'
balanced_df['iso2_code'] = balanced_df['unique_id'].str.extract(r'^([A-Z]{2})-')[0]
balanced_df['language'] = balanced_df['unique_id'].str.split('-').str[1:].str.join('-')
balanced_df[['year', 'quarter']] = balanced_df['year_quarter'].str.split('-', expand=True) # AGREGADO
quarter_map = {'Q1': 1, 'Q2': 2, 'Q3': 3, 'Q4': 4} # AGREGADO
balanced_df['quarter'] = balanced_df['quarter'].map(quarter_map).astype(int) # AGREGADO
balanced_df['year'] = balanced_df['year'].astype(int) # AGREGADO
balanced_df

,unique_id,year_quarter,num_pushers,iso2_code,language,year,quarter
0,AE-HTML,2020-Q1,2066.0,AE,HTML,2020,1
1,AE-HTML,2020-Q2,3254.0,AE,HTML,2020,2
2,AE-HTML,2020-Q3,3208.0,AE,HTML,2020,3
3,AE-HTML,2020-Q4,3255.0,AE,HTML,2020,4
4,AE-HTML,2021-Q1,3321.0,AE,HTML,2021,1
...,...,...,...,...,...,...,...
136203,YE-TypeScript,2022-Q4,0.0,YE,TypeScript,2022,4
136204,YE-TypeScript,2023-Q1,0.0,YE,TypeScript,2023,1
136205,YE-TypeScript,2023-Q2,0.0,YE,TypeScript,2023,2
136206,YE-TypeScript,2023-Q3,0.0,YE,TypeScript,2023,3


In [54]:
# Function to convert quarter format to integer
def quarter_to_int(quarter_string):
    year, q = quarter_string.split('-')
    year = int(year)
    quarter_number = int(q[1])  # Q1, Q2, Q3, Q4 -> 1, 2, 3, 4
    base_year = 2020  # Adjust based on your balanced_df, or set dynamically
    return 4 * (year - base_year) + quarter_number

# Applying the function
balanced_df['quarter_int'] = balanced_df['year_quarter'].apply(quarter_to_int)
balanced_df

,unique_id,year_quarter,num_pushers,iso2_code,language,year,quarter,quarter_int
0,AE-HTML,2020-Q1,2066.0,AE,HTML,2020,1,1
1,AE-HTML,2020-Q2,3254.0,AE,HTML,2020,2,2
2,AE-HTML,2020-Q3,3208.0,AE,HTML,2020,3,3
3,AE-HTML,2020-Q4,3255.0,AE,HTML,2020,4,4
4,AE-HTML,2021-Q1,3321.0,AE,HTML,2021,1,5
...,...,...,...,...,...,...,...,...
136203,YE-TypeScript,2022-Q4,0.0,YE,TypeScript,2022,4,12
136204,YE-TypeScript,2023-Q1,0.0,YE,TypeScript,2023,1,13
136205,YE-TypeScript,2023-Q2,0.0,YE,TypeScript,2023,2,14
136206,YE-TypeScript,2023-Q3,0.0,YE,TypeScript,2023,3,15


## Per capita measures

In [55]:
# Definimos una función que convierte los nombres de países en códigos iso2
def country_to_iso2(country_name):
    try:
        # Intentamos obtener el código iso2 del país usando pycountry
        return pycountry.countries.get(name=country_name).alpha_2
    except AttributeError:
        try:
            # Manejamos casos especiales donde el nombre del país no coincide exactamente con la base de datos de pycountry
            special_cases = {
                "Czechia (Czech Republic)": "CZ",
                "Congo (Congo-Brazzaville)": "CG",
                "Holy See": "VA",
                "Timor-Leste (East Timor)": "TL",
                "Ukraine (with certain exceptions)": "UA",
                "Taiwan": "TW",
                "Bolivia": "BO",
                "Tanzania": "TZ",
                "South Korea": "KR",
                "Moldova": "MD",
                "Brunei": "BN"
            }
            return special_cases[country_name]
        except KeyError:
            return None

# Creamos una lista de países y obtenemos sus códigos iso2 usando la función country_to_iso2
gpt_countries_list = [
    "Albania", "Algeria", "Andorra", "Angola", "Antigua and Barbuda", "Argentina", "Armenia", "Australia", "Austria",
    "Azerbaijan", "Bahamas", "Bangladesh", "Barbados", "Belgium", "Belize", "Benin", "Bhutan", "Bolivia",
    "Bosnia and Herzegovina", "Botswana", "Brazil", "Brunei", "Bulgaria", "Burkina Faso", "Cabo Verde", "Canada",
    "Chile", "Colombia", "Comoros", "Congo (Congo-Brazzaville)", "Costa Rica", "Côte d'Ivoire", "Croatia", "Cyprus",
    "Czechia", "Denmark", "Djibouti", "Dominica", "Dominican Republic", "Ecuador", "El Salvador", "Estonia", "Fiji",
    "Finland", "France", "Gabon", "Gambia", "Georgia", "Germany", "Ghana", "Greece", "Grenada", "Guatemala", "Guinea",
    "Guinea-Bissau", "Guyana", "Haiti", "Holy See", "Honduras", "Hungary", "Iceland", "India", "Indonesia", "Iraq",
    "Ireland", "Israel", "Italy", "Jamaica", "Japan", "Jordan", "Kazakhstan", "Kenya", "Kiribati", "Kuwait",
    "Kyrgyzstan", "Latvia", "Lebanon", "Lesotho", "Liberia", "Liechtenstein", "Lithuania", "Luxembourg", "Madagascar",
    "Malawi", "Malaysia", "Maldives", "Mali", "Malta", "Marshall Islands", "Mauritania", "Mauritius", "Mexico",
    "Micronesia", "Moldova", "Monaco", "Mongolia", "Montenegro", "Morocco", "Mozambique", "Myanmar", "Namibia",
    "Nauru", "Nepal", "Netherlands", "New Zealand", "Nicaragua", "Niger", "Nigeria", "North Macedonia", "Norway",
    "Oman", "Pakistan", "Palau", "Palestine, State of", "Panama", "Papua New Guinea", "Paraguay", "Peru", "Philippines",
    "Poland", "Portugal", "Qatar", "Romania", "Rwanda", "Saint Kitts and Nevis", "Saint Lucia",
    "Saint Vincent and the Grenadines", "Samoa", "San Marino", "Sao Tome and Principe", "Saudi Arabia", "Senegal",
    "Serbia", "Seychelles", "Sierra Leone", "Singapore", "Slovakia", "Slovenia", "Solomon Islands", "South Africa",
    "South Korea", "Spain", "Sri Lanka", "Suriname", "Sweden", "Switzerland", "Taiwan", "Tanzania", "Thailand",
    "Timor-Leste", "Togo", "Tonga", "Trinidad and Tobago", "Tunisia", "Turkey", "Tuvalu", "Uganda", "Ukraine",
    "United Arab Emirates", "United Kingdom", "United States", "Uruguay", "Vanuatu", "Zambia"
]

gpt_countries_iso = [country_to_iso2(country) for country in gpt_countries_list]

# Agregamos una nueva columna gpt_available, con valor 1 si el código iso2 está en la lista gpt_countries_iso, y 0 en caso contrario
balanced_df["gpt_available"] = balanced_df["iso2_code"].apply(lambda row: 1 if row in gpt_countries_iso else 0)

In [56]:
countries = balanced_df.iso2_code.unique()

def create_populations_dictionary():
    country_populations = {}
    special_cases = {"MM": 54688774, "PS": 5483450, "ME": 602445, "AD":79824}
    # special_cases = {}
    for country in countries:
        try:
            country_populations.update({country: CInfo(country).info()["population"]})
        except KeyError:
            try:
                fallback_name = pycountry.countries.lookup(country).name
                country_populations.update({country: CInfo(fallback_name).info()["population"]})
            except KeyError:
                print(country)
                country_populations.update({country: special_cases[country]})

    return country_populations

country_populations = create_populations_dictionary()

ME
MM
PS
AD


# # New variables

In [57]:
# Create population
balanced_df["population"] = balanced_df["iso2_code"].map(country_populations)

# Create pushes_pc
balanced_df["num_pushers_pc"] = (balanced_df["num_pushers"] / balanced_df["population"])*100000

# Create post_CI
balanced_df['post_CI'] = ((balanced_df['quarter_int'] >= 12) & (balanced_df['gpt_available'] == 1)).astype(int)

# Create gtp_available_post1 and gtp_available_post2
# balanced_df['gpt_available_post1'] = balanced_df['gpt_available'] * balanced_df['post_1']
#balanced_df['gpt_available_post2'] = balanced_df['gpt_available'] * balanced_df['post_2']

In [58]:
balanced_df

,unique_id,year_quarter,num_pushers,iso2_code,language,year,quarter,quarter_int,gpt_available,population,num_pushers_pc,post_CI
0,AE-HTML,2020-Q1,2066.0,AE,HTML,2020,1,1,1,9446000,21.871692,0
1,AE-HTML,2020-Q2,3254.0,AE,HTML,2020,2,2,1,9446000,34.448444,0
2,AE-HTML,2020-Q3,3208.0,AE,HTML,2020,3,3,1,9446000,33.961465,0
3,AE-HTML,2020-Q4,3255.0,AE,HTML,2020,4,4,1,9446000,34.459030,0
4,AE-HTML,2021-Q1,3321.0,AE,HTML,2021,1,5,1,9446000,35.157739,0
...,...,...,...,...,...,...,...,...,...,...,...,...
136203,YE-TypeScript,2022-Q4,0.0,YE,TypeScript,2022,4,12,0,25956000,0.000000,0
136204,YE-TypeScript,2023-Q1,0.0,YE,TypeScript,2023,1,13,0,25956000,0.000000,0
136205,YE-TypeScript,2023-Q2,0.0,YE,TypeScript,2023,2,14,0,25956000,0.000000,0
136206,YE-TypeScript,2023-Q3,0.0,YE,TypeScript,2023,3,15,0,25956000,0.000000,0


In [59]:
balanced_df.to_csv("D:/JetBrains/Projects/pythonProject/balanced_df.csv")

In [8]:
df = pd.read_csv("balanced_df.csv")
df = df.drop('Unnamed: 0', axis=1)
df

,unique_id,year_quarter,num_pushers,iso2_code,language,year,quarter,quarter_int,gpt_available,population,num_pushers_pc,post_CI
0,AE-HTML,2020-Q1,2066.0,AE,HTML,2020,1,1,1,9446000,21.871692,0
1,AE-HTML,2020-Q2,3254.0,AE,HTML,2020,2,2,1,9446000,34.448444,0
2,AE-HTML,2020-Q3,3208.0,AE,HTML,2020,3,3,1,9446000,33.961465,0
3,AE-HTML,2020-Q4,3255.0,AE,HTML,2020,4,4,1,9446000,34.459030,0
4,AE-HTML,2021-Q1,3321.0,AE,HTML,2021,1,5,1,9446000,35.157739,0
...,...,...,...,...,...,...,...,...,...,...,...,...
136203,YE-TypeScript,2022-Q4,0.0,YE,TypeScript,2022,4,12,0,25956000,0.000000,0
136204,YE-TypeScript,2023-Q1,0.0,YE,TypeScript,2023,1,13,0,25956000,0.000000,0
136205,YE-TypeScript,2023-Q2,0.0,YE,TypeScript,2023,2,14,0,25956000,0.000000,0
136206,YE-TypeScript,2023-Q3,0.0,YE,TypeScript,2023,3,15,0,25956000,0.000000,0


In [9]:
df.isnull().sum()

unique_id          0
year_quarter       0
num_pushers        0
iso2_code         64
language           0
year               0
quarter            0
quarter_int        0
gpt_available      0
population         0
num_pushers_pc     0
post_CI            0
dtype: int64

# # DataFrame Original

In [5]:
all_languages_panel = pd.read_csv("all_languages_panel.csv")
all_languages_panel = all_languages_panel.drop('Unnamed: 0', axis=1)
all_languages_panel

,unique_id,year_quarter,num_pushers,iso2_code,language,year,quarter,quarter_int,gpt_available,population,num_pushers_pc,post_CI,python,Treatment,post_1,post_2,gpt_available_post1,gpt_available_post2,Category
0,AE-HTML,2020-Q1,2066.0,AE,HTML,2020,1,1,1,9446000,21.871692,0,0,0,0,0,0,0,Domain-Specific Languages (DSLs)
1,AE-HTML,2020-Q2,3254.0,AE,HTML,2020,2,2,1,9446000,34.448444,0,0,0,0,0,0,0,Domain-Specific Languages (DSLs)
2,AE-HTML,2020-Q3,3208.0,AE,HTML,2020,3,3,1,9446000,33.961465,0,0,0,0,0,0,0,Domain-Specific Languages (DSLs)
3,AE-HTML,2020-Q4,3255.0,AE,HTML,2020,4,4,1,9446000,34.459030,0,0,0,0,0,0,0,Domain-Specific Languages (DSLs)
4,AE-HTML,2021-Q1,3321.0,AE,HTML,2021,1,5,1,9446000,35.157739,0,0,0,0,0,0,0,Domain-Specific Languages (DSLs)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136203,YE-TypeScript,2022-Q4,0.0,YE,TypeScript,2022,4,12,0,25956000,0.000000,0,0,0,1,0,0,0,"High-Level, General-Purpose Languages"
136204,YE-TypeScript,2023-Q1,0.0,YE,TypeScript,2023,1,13,0,25956000,0.000000,0,0,0,1,1,0,0,"High-Level, General-Purpose Languages"
136205,YE-TypeScript,2023-Q2,0.0,YE,TypeScript,2023,2,14,0,25956000,0.000000,1,0,0,1,1,0,0,"High-Level, General-Purpose Languages"
136206,YE-TypeScript,2023-Q3,0.0,YE,TypeScript,2023,3,15,0,25956000,0.000000,1,0,0,1,1,0,0,"High-Level, General-Purpose Languages"


In [7]:
all_languages_panel.isnull().sum()

unique_id               0
year_quarter            0
num_pushers             0
iso2_code              64
language                0
year                    0
quarter                 0
quarter_int             0
gpt_available           0
population              0
num_pushers_pc          0
post_CI                 0
python                  0
Treatment               0
post_1                  0
post_2                  0
gpt_available_post1     0
gpt_available_post2     0
Category                0
dtype: int64

In [12]:
all_languages_panel['post_CI'].value_counts()

post_CI
0    110669
1     25539
Name: count, dtype: int64

In [11]:
df['post_CI'].value_counts()

post_CI
0    99938
1    36270
Name: count, dtype: int64